In [3]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model

In [4]:
user = pd.read_csv('./data/user.csv')
destination = pd.read_csv('./data/destinasi-wisata-indonesia.csv')
bali =  pd.read_csv('./data/Dataset - tourismBali.csv')

In [5]:
user.head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


In [43]:
bali.head()

,Place_Id,Place_Name,Description,Weekend Holiday Price,Weekday Price,Category,City,Rating,Alamat,Coordinate,Lat,Long,Gambar
0,1,Agrowisata Satria,Agrowisata Satria menawarkan 'rasa' Bali pada ...,50000,50000,Agrowisata,Gianyar,3.0,"Manukaya, Kec. Tampaksiring, Kabupaten Gianya...","-8.402228546131536, 115.32296693687739",-8.402229,115.322967,V
1,2,Agung Bali,Dapatkan berbagai produk oleh-oleh khas Bali b...,0,0,Belanja,Badung,4.0,"Jln. Dewi Sri No.18XX, Kuta, Badung, Bali","-8.700234336021559, 115.176534407375",-8.700234,115.176534,V
2,3,Wisata Air Panas Toya Bungkah,Pemandian air panas di dekat gunung Batur (Pem...,70000,70000,Alam,Bangli,4.0,"Batur Tengah, Kec. Kintamani, Kabupaten Bangli...","-8.251298580809106, 115.39981444195475",-8.251299,115.399814,V
3,4,Air Terjun Aling-Aling,Keberadaan air terjun ini semakin mempercantik...,20000,20000,Alam,Buleleng,4.5,"Jl. Raya Desa Sambangan, Banjar, Sambangan, Ke...","-8.173073590748519, 115.10513024047802",-8.173074,115.105130,V
4,5,Air Terjun Tegenungan,Bosan dengan wisata alam pantai dan ingin meni...,20000,20000,Alam,Gianyar,4.0,"Jl. Ir. Sutami, Kemenuh, Kec. Sukawati, Kabupa...","-8.575191460385602, 115.28870189592831",-8.575191,115.288702,V


In [44]:
bali[['Weekday Price', 'Weekend Holiday Price']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Weekday Price          75 non-null     int64
 1   Weekend Holiday Price  75 non-null     int64
dtypes: int64(2)
memory usage: 1.3 KB


In [49]:
mean = (bali['Weekday Price'].values + bali['Weekend Holiday Price'].values) / 2

In [50]:
mean

array([ 50000.,      0.,  70000.,  20000.,  20000.,  15000., 300000.,
       102500.,  20000., 500000.,      0.,  32500.,  50000.,      0.,
        15000.,  30000.,  10000.,  10000.,      0.,      0.,      0.,
        95000.,  30000.,  25000.,  15000.,  10000.,      0.,  40000.,
            0.,      0.,  25000.,      0.,  60000.,  25000.,  25000.,
        10000.,  10000.,      0.,      0.,  10000.,  10000.,      0.,
            0.,   5000.,      0.,   8000.,      0.,   5000.,   8000.,
            0.,      0.,  10000.,      0.,      0.,      0.,      0.,
        30000.,  30000.,  10000.,  50000.,  15000.,      0.,  15000.,
        15000.,  30000.,  50000.,      0.,      0., 100000., 150000.,
        10000.,  25000., 289000.,  10000.,      0.])

In [51]:
bali['Price'] = mean

In [52]:
bali

,Place_Id,Place_Name,Description,Weekend Holiday Price,Weekday Price,Category,City,Rating,Alamat,Coordinate,Lat,Long,Gambar,Price
0,1,Agrowisata Satria,Agrowisata Satria menawarkan 'rasa' Bali pada ...,50000,50000,Agrowisata,Gianyar,3.0,"Manukaya, Kec. Tampaksiring, Kabupaten Gianya...","-8.402228546131536, 115.32296693687739",-8.402229,115.322967,V,50000.0
1,2,Agung Bali,Dapatkan berbagai produk oleh-oleh khas Bali b...,0,0,Belanja,Badung,4.0,"Jln. Dewi Sri No.18XX, Kuta, Badung, Bali","-8.700234336021559, 115.176534407375",-8.700234,115.176534,V,0.0
2,3,Wisata Air Panas Toya Bungkah,Pemandian air panas di dekat gunung Batur (Pem...,70000,70000,Alam,Bangli,4.0,"Batur Tengah, Kec. Kintamani, Kabupaten Bangli...","-8.251298580809106, 115.39981444195475",-8.251299,115.399814,V,70000.0
3,4,Air Terjun Aling-Aling,Keberadaan air terjun ini semakin mempercantik...,20000,20000,Alam,Buleleng,4.5,"Jl. Raya Desa Sambangan, Banjar, Sambangan, Ke...","-8.173073590748519, 115.10513024047802",-8.173074,115.105130,V,20000.0
4,5,Air Terjun Tegenungan,Bosan dengan wisata alam pantai dan ingin meni...,20000,20000,Alam,Gianyar,4.0,"Jl. Ir. Sutami, Kemenuh, Kec. Sukawati, Kabupa...","-8.575191460385602, 115.28870189592831",-8.575191,115.288702,V,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,Teras Sawah Tegalalang,"Uniknya Tegallalang Rice Terrace di Bali ini, ...",10000,10000,Agrowisata,Gianyar,4.0,"Jl. Raya Tegallalang, Tegallalang, Kec. Tegall...","-8.433891714602193, 115.27918179571854",-8.433892,115.279182,V,10000.0
71,72,Tirta Gangga,Ada tiga hal utama yang ada di taman Tirta Gan...,25000,25000,Religius,Karangasem,4.5,"Jalan Raya Abang Desa Adat, Ababi, Kec. Abang,...","-8.412071345387545, 115.58728116872855",-8.412071,115.587281,V,25000.0
72,73,Waterbom Bali,Waterbom merupakan sebuah oase di jantung kawa...,289000,289000,Rekreasi,Badung,4.5,"Jl. Kartika Plaza, Tuban, Kec. Kuta, Kabupaten...","-8.72828817001415, 115.16934211106027",-8.728288,115.169342,V,289000.0
73,74,Alas Harum Agro Tourism,Alas Harum Bali merupakan destinasi wisata yan...,10000,10000,Agrowisata,Gianyar,4.0,"Jl. Raya Tegallalang, Tegallalang, Kec. Tegall...","-8.43969896624251, 115.28060749756382",-8.439699,115.280608,V,10000.0


In [6]:
destination.shape

(437, 14)

In [7]:
destination.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Column1,_1,Rating_Count
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,46,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-61753924,1068271528,NaN,1,18
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,46,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-61376448,1068171245,NaN,2,25
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,46,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-61253124,1068335377,NaN,3,19
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,45,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-63024459,1068951559,NaN,4,21
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,45,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-612419,106839134,NaN,5,24


In [8]:
destination['City'].value_counts()

City
Yogyakarta    126
Bandung       124
Jakarta        84
Semarang       57
Surabaya       46
Name: count, dtype: int64

In [9]:
destination.isnull().sum()

Place_Id          0
Place_Name        0
Description       0
Category          0
City              0
Price             0
Rating            0
Time_Minutes    232
Coordinate        0
Lat               0
Long              0
Column1         437
_1                0
Rating_Count      0
dtype: int64

In [10]:
destination.drop(columns=['Time_Minutes', 'Column1', '_1'], inplace=True)

In [11]:
destination.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Coordinate', 'Lat', 'Long', 'Rating_Count'],
      dtype='object')

In [12]:
destination.duplicated().sum()

0

In [13]:
destination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Place_Id      437 non-null    int64 
 1   Place_Name    437 non-null    object
 2   Description   437 non-null    object
 3   Category      437 non-null    object
 4   City          437 non-null    object
 5   Price         437 non-null    int64 
 6   Rating        437 non-null    int64 
 7   Coordinate    437 non-null    object
 8   Lat           437 non-null    int64 
 9   Long          437 non-null    int64 
 10  Rating_Count  437 non-null    int64 
dtypes: int64(6), object(5)
memory usage: 37.7+ KB


##REKOMENDASI BY KATEGORI WISATA

### PREPROCESSING
di encode + cari kesamaan pake cosine similarity

In [14]:
oneHotCategory = OneHotEncoder(sparse=False)
oneHotCity = OneHotEncoder(sparse=False)

In [15]:
categoryEncoded = oneHotCategory.fit_transform(destination[['Category']])
cityEncoded = oneHotCity.fit_transform(destination[['City']])

c:\Users\Akmal Fauzan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Akmal Fauzan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
categoryEncoded = pd.DataFrame(categoryEncoded, columns=oneHotCategory.get_feature_names_out(['Category']))
cityEncoded = pd.DataFrame(cityEncoded, columns=oneHotCity.get_feature_names_out(['City']))

In [17]:
categoryEncoded

,Category_Bahari,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Taman Hiburan,Category_Tempat Ibadah
0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
432,0.0,1.0,0.0,0.0,0.0,0.0
433,0.0,0.0,0.0,0.0,1.0,0.0
434,0.0,0.0,0.0,0.0,1.0,0.0
435,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
cityEncoded

,City_Bandung,City_Jakarta,City_Semarang,City_Surabaya,City_Yogyakarta
0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...
432,0.0,0.0,0.0,1.0,0.0
433,0.0,0.0,0.0,1.0,0.0
434,0.0,0.0,0.0,1.0,0.0
435,0.0,0.0,0.0,1.0,0.0


In [19]:
cosineSim = cosine_similarity(categoryEncoded)
cosineCity = cosine_similarity(cityEncoded)

In [20]:
index_combined = destination['Place_Name'] + " - " + destination['City']
cosineSim = pd.DataFrame(cosineSim, index=index_combined, columns=index_combined)
cosineCity = pd.DataFrame(cosineCity, index=index_combined, columns=index_combined)

In [21]:
cosineSim

,Monumen Nasional - Jakarta,Kota Tua - Jakarta,Dunia Fantasi - Jakarta,Taman Mini Indonesia Indah (TMII) - Jakarta,Atlantis Water Adventure - Jakarta,Taman Impian Jaya Ancol - Jakarta,Kebun Binatang Ragunan - Jakarta,Ocean Ecopark - Jakarta,Pelabuhan Marina - Jakarta,Pulau Tidung - Jakarta,...,Monumen Bambu Runcing Surabaya - Surabaya,House of Sampoerna - Surabaya,Atlantis Land Surabaya - Surabaya,Taman Hiburan Rakyat - Surabaya,Taman Mundu - Surabaya,Museum Mpu Tantular - Surabaya,Taman Bungkul - Surabaya,Taman Air Mancur Menari Kenjeran - Surabaya,Taman Flora Bratang Surabaya - Surabaya,Gereja Perawan Maria Tak Berdosa Surabaya - Surabaya
Monumen Nasional - Jakarta,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Kota Tua - Jakarta,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Dunia Fantasi - Jakarta,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
Taman Mini Indonesia Indah (TMII) - Jakarta,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
Atlantis Water Adventure - Jakarta,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Museum Mpu Tantular - Surabaya,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Taman Bungkul - Surabaya,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
Taman Air Mancur Menari Kenjeran - Surabaya,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
Taman Flora Bratang Surabaya - Surabaya,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0


In [22]:
cosineCity

,Monumen Nasional - Jakarta,Kota Tua - Jakarta,Dunia Fantasi - Jakarta,Taman Mini Indonesia Indah (TMII) - Jakarta,Atlantis Water Adventure - Jakarta,Taman Impian Jaya Ancol - Jakarta,Kebun Binatang Ragunan - Jakarta,Ocean Ecopark - Jakarta,Pelabuhan Marina - Jakarta,Pulau Tidung - Jakarta,...,Monumen Bambu Runcing Surabaya - Surabaya,House of Sampoerna - Surabaya,Atlantis Land Surabaya - Surabaya,Taman Hiburan Rakyat - Surabaya,Taman Mundu - Surabaya,Museum Mpu Tantular - Surabaya,Taman Bungkul - Surabaya,Taman Air Mancur Menari Kenjeran - Surabaya,Taman Flora Bratang Surabaya - Surabaya,Gereja Perawan Maria Tak Berdosa Surabaya - Surabaya
Monumen Nasional - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kota Tua - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dunia Fantasi - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taman Mini Indonesia Indah (TMII) - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Atlantis Water Adventure - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Museum Mpu Tantular - Surabaya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Taman Bungkul - Surabaya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Taman Air Mancur Menari Kenjeran - Surabaya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Taman Flora Bratang Surabaya - Surabaya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Content-Based

In [23]:
def getRecommendationsByCategory(category, destination, similarity, top_n=5):
    relevant_places = destination[destination['Category'] == category]

    relevant_indices = relevant_places['Place_Name'] + " - " + relevant_places['City']

    if relevant_indices.empty:
        return "Tidak ada tempat wisata dengan kategori yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = similarity[index]

        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores

    return recommendations

In [24]:
recommendationsByCategory = getRecommendationsByCategory("Monumen", destination, cosineSim, top_n=5)
print("Rekomendasi Berdasarkan Kategori Wisata:")
print(recommendationsByCategory)

Rekomendasi Berdasarkan Kategori Wisata:
Tidak ada tempat wisata dengan kategori yang diminta.


In [25]:
def getRecommendationsByCity(city, destination, similarity, top_n=5):
    relevant_places = destination[destination['City'] == city]

    relevant_indices = relevant_places['Place_Name'] + " - " + relevant_places['City']

    if relevant_indices.empty:
        return "Tidak ada tempat wisata di kota yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = similarity[index]
        
        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores

    return recommendations

In [26]:
recommendationsByCity = getRecommendationsByCity("Jakarta", destination, cosineCity, top_n=5)
print("\nRekomendasi Berdasarkan Kota:")
print(recommendationsByCity)


Rekomendasi Berdasarkan Kota:
{'Monumen Nasional - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Monumen Nasional - Jakarta, dtype: float64, 'Kota Tua - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Kota Tua - Jakarta, dtype: float64, 'Dunia Fantasi - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Dunia Fantasi - Jakarta, dtype: float64, 'Taman Mini Indonesia Indah (TMII) - Jakarta': Klenteng Jin De Yuan - Jakar

In [27]:
weightCategory = 0.5  
weightCity = 0.5

combinedSimilarity = (weightCategory * cosineSim) + (weightCity * cosineCity)

In [28]:
def getCombinedRecommendations(category, city, destination, combined_similarity, top_n=5):
    relevant_places = destination[(destination['City'] == city) & 
                                     (destination['Category'] == category)]
    
    relevant_indices = relevant_places['Place_Name'] + " - " + relevant_places['City']
    
    if relevant_indices.empty:
        return "Tidak ada tempat yang cocok dengan kategori dan kota yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = combined_similarity[index]
        
        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores
    
    return recommendations


In [29]:
budaya = destination[(destination['Category'] == 'Budaya') & (destination['City'] == 'Yogyakarta')]

In [30]:
budaya

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Coordinate,Lat,Long,Rating_Count
85,86,Keraton Yogyakarta,Keraton Ngayogyakarta Hadiningrat atau Keraton...,Budaya,Yogyakarta,15000,46,"{'lat': -7.8052845, 'lng': 110.3642031}",-78052845,1103642031,24
87,88,Museum Benteng Vredeburg Yogyakarta,Museum Benteng Vredeburg (bahasa Jawa: ꦩꦸꦱꦶꦪꦸꦩ...,Budaya,Yogyakarta,3000,46,"{'lat': -7.800201599999999, 'lng': 110.3663044}",-78002016,1103663044,32
88,89,De Mata Museum Jogja,Museum De Mata merupakan salah satu museum yan...,Budaya,Yogyakarta,50000,44,"{'lat': -7.816315599999999, 'lng': 110.3871442}",-78163156,1103871442,30
96,97,Monumen Yogya Kembali,Museum Monumen Yogya Kembali (bahasa Jawa: ꦩꦺꦴ...,Budaya,Yogyakarta,15000,45,"{'lat': -7.7495904, 'lng': 110.3696068}",-77495904,1103696068,22
98,99,Kampung Wisata Kadipaten,Kampung Wisata Kadipaten secaara kewilayahan b...,Budaya,Yogyakarta,0,44,"{'lat': -7.806093000000001, 'lng': 110.35831}",-7806093,11035831,24
99,100,Taman Budaya Yogyakarta,Taman Budaya Yogyakarta (TBY) (Hanacaraka:ꦠꦩꦤ꧀...,Budaya,Yogyakarta,0,45,"{'lat': -7.8001041, 'lng': 110.3676579}",-78001041,1103676579,20
100,101,Kampung Wisata Sosro Menduran,Kampung wisata Sosromenduran merupakan kampung...,Budaya,Yogyakarta,0,4,"{'lat': -7.792189999999999, 'lng': 110.362151}",-779219,110362151,23
101,102,Monumen Batik Yogyakarta,Perhatian pemerintah megenai kebudayan batik i...,Budaya,Yogyakarta,40000,45,"{'lat': -7.8011159, 'lng': 110.3646172}",-78011159,1103646172,24
103,104,Tebing Breksi,Tebing Breksi merupakan tempat wisata yang ber...,Budaya,Yogyakarta,20000,44,"{'lat': -7.781571400000001, 'lng': 110.5045827}",-77815714,1105045827,22
104,105,Candi Donotirto,"Walaupun melekat dengan kata candi, Candi Dono...",Budaya,Yogyakarta,0,44,"{'lat': -7.795561299999998, 'lng': 110.3612095}",-77955613,1103612095,19


In [31]:
# Buat rekomendasi berdasarkan kategori wisata dan kota
recommendations_combined = getCombinedRecommendations("Budaya", "Yogyakarta", destination, combinedSimilarity, top_n=5)
print("Rekomendasi Berdasarkan Kategori dan Kota:")
print(recommendations_combined)


Rekomendasi Berdasarkan Kategori dan Kota:
{'Keraton Yogyakarta - Yogyakarta': Tebing Breksi - Yogyakarta              1.0
Monumen Sanapati - Yogyakarta           1.0
Gedung Agung Yogyakarta - Yogyakarta    1.0
Candi Borobudur - Yogyakarta            1.0
Wisata Kraton Jogja - Yogyakarta        1.0
Name: Keraton Yogyakarta - Yogyakarta, dtype: float64, 'Museum Benteng Vredeburg Yogyakarta - Yogyakarta': Tebing Breksi - Yogyakarta              1.0
Monumen Sanapati - Yogyakarta           1.0
Gedung Agung Yogyakarta - Yogyakarta    1.0
Candi Borobudur - Yogyakarta            1.0
Wisata Kraton Jogja - Yogyakarta        1.0
Name: Museum Benteng Vredeburg Yogyakarta - Yogyakarta, dtype: float64, 'De Mata Museum Jogja - Yogyakarta': Tebing Breksi - Yogyakarta              1.0
Monumen Sanapati - Yogyakarta           1.0
Gedung Agung Yogyakarta - Yogyakarta    1.0
Candi Borobudur - Yogyakarta            1.0
Wisata Kraton Jogja - Yogyakarta        1.0
Name: De Mata Museum Jogja - Yogyakarta, dtyp

### Collaborative Filtering

In [32]:
num_users = 1000  
num_places = len(destination)  
embedding_dim = 50

In [33]:
user_input = Input(shape=(1,), name='user_input')
place_input = Input(shape=(1,), name='place_input')

In [34]:
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(user_input)
place_embedding = Embedding(input_dim=num_places, output_dim=embedding_dim, name='place_embedding')(place_input)

In [35]:
user_vec = Flatten()(user_embedding)
place_vec = Flatten()(place_embedding)

In [36]:
similarityScores = Dot(axes=1)([user_vec, place_vec])

In [37]:
model = Model(inputs=[user_input, place_input], outputs=similarityScores)
model.compile(optimizer='adam', loss='mse')

In [38]:
user_ids = np.array([1, 2, 3, 4])  
place_ids = np.array([10, 20, 30, 40])  
ratings = np.array([4.0, 5.0, 3.0, 4.5])

In [39]:
model.fit([user_ids, place_ids], ratings, epochs=10, verbose=1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 17.5718
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 17.5516
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 17.5314
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 17.5113
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 17.4911
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 17.4709
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 17.4506
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 17.4302
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 17.4096
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 17.3889


In [40]:
user_id = np.array([1])  
place_id = np.array([10])

In [41]:
predicted_score = model.predict([user_id, place_id])
print(f"skor prediksi user {user_id[0]} dan tempat {place_id[0]}: {predicted_score[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
skor prediksi user 1 dan tempat 10: 0.026265759021043777
